In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [2]:
import requests

In [3]:
import requests

In [4]:
main_url = 'https://www.fxp.co.il/forumdisplay.php?f=46'

In [5]:
response = requests.get(main_url)

In [6]:
contents = response.content.decode("utf-8")

In [7]:
response.close()

In [8]:
import re

In [9]:
b'<a class="title" href="showthread.php?t=20164299" id="thread_title_20164299">'

b'<a class="title" href="showthread.php?t=20164299" id="thread_title_20164299">'

In [10]:
print(contents)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" dir="rtl" lang="he" id="vbulletin_html">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta id="e_vb_meta_bburl" name="vb_meta_bburl" content="https://www.fxp.co.il" />
<meta http-equiv="X-UA-Compatible" content="IE=100" />
<base href="//www.fxp.co.il" /><!--[if IE]></base><![endif]-->
<link rel="canonical" href="https://www.fxp.co.il/forumdisplay.php?f=46" />
<link rel="shortcut icon" href="favicon.ico">
<link rel="apple-touch-icon" href="//static.fcdn.co.il/css_static_main/logo512.png" />
<meta name="generator" content="vBulletin 4.2.3" />
<meta name="keywords" content="צבא וביטחון, השתמטות, לוחמים, חטיבה, הורדת פרופיל, לקנות לצבא, יחידות התנדבותיות, 8200, אוגדה, טירונים,להיות לוחם" />
<meta name="description" content="כאן המקום לדבר על כל דבר ה

In [11]:
all_threads = re.findall("showthread\.php\?t=(.*)\" id=", contents)

In [12]:
all_threads[-1]

'20165623'

In [13]:
thread_url = '20164299'

In [14]:
print(thread_url)

20164299


In [15]:
url_to_page = 'https://www.fxp.co.il/showthread.php?t=%s&page=%d'%(thread_url,2)

In [16]:
url_to_page = 'https://www.fxp.co.il/showthread.php?t=20165100&page=2'

In [17]:
response_page = requests.get(url_to_page)

In [18]:
content_page = response_page.content.decode("utf-8")

In [19]:
if response_page.history:
    print("Request was redirected")
    for resp in response_page.history:
        print (resp.status_code, resp.url)
    print("Final destination:")
    print( response_page.status_code, response_page.url)
else:
    print("Request was not redirected")

Request was not redirected


In [20]:
content_page

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="https://www.facebook.com/2008/fbml" dir="rtl" lang="he" id="vbulletin_html">\n<head>\n<meta itemprop="datePublished" content="2020-03-16">\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<meta id="e_vb_meta_bburl" name="vb_meta_bburl" content="https://www.fxp.co.il" />\n<meta http-equiv="X-UA-Compatible" content="IE=100" />\n<base href="//www.fxp.co.il" /><!--[if IE]></base><![endif]-->\n<link rel="shortcut icon" href="favicon.ico">\n<link rel="apple-touch-icon" href="//static.fcdn.co.il/css_static_main/logo512.png" />\n<meta name="generator" content="vBulletin 4.2.3" />\n<meta property="fb:app_id" content="415294715208536" />\n<meta property="og:site_name" content="FXP" />\n<meta property="og:description" content="מלש&quot;ב תמים שלא חווה ר

In [21]:
re.findall("itemprop", content_page)

['itemprop', 'itemprop', 'itemprop']

In [22]:
import codecs
file = codecs.open("m.html", "w", "utf-8")
file.write(content_page)
file.close()

In [29]:
from datetime import datetime
import pandas as pd


In [35]:
def thread_ids(pages, verbose=True):
    print('Fetching pages...')
    count = 0
    base_main_url = 'https://www.fxp.co.il/forumdisplay.php?f=46&pages=%d'
    thread_ids = []
    for i in range(pages):
        main_url = base_main_url % i
        response = requests.get(main_url)
        contents = response.content.decode("utf-8")
        response.close()
        currenct_thread_ids = re.findall("showthread\.php\?t=(.*)\" id=", contents)
        thread_ids +=  [int(x) for x in currenct_thread_ids]
        count += 1
        if verbose and (count % 10 == 0):
            print('Fetched %s pages' % (count))
            print(datetime.now().time())
    print(thread_ids)
    return thread_ids

In [38]:
def thread_content(thread_ids, file, verbose=True):
    print('Fetching threads...')
    count = 0
    df = pd.DataFrame(columns=['thread', 'post'])
    for i in thread_ids:
        request = Request(
            url='https://www.fxp.co.il/showthread.php?t=%s' %
            (i),
            headers=headers)
        tree = html.fromstring(urlopen(request).read().decode('utf-8'))
        title = tree.xpath('//div[@class=\'titleshowt greengr\']/h1/text()')
        df.append({'thread': i, 'post': title[0]},
                  ignore_index=True)
        messages = tree.xpath('//div[@id=\'postlist\']//blockquote\
            [@class=\'postcontent restore\']')
        for m in messages:
            message_text = ' '.join(m.xpath('./text()'))
            df = df.append({'thread': i, 'post': message_text},
                           ignore_index=True)
        last_page = tree.xpath('//span[@class=\'first_last\']/a/@href')
        if len(last_page) > 0:
            last_page = last_page[0]
            last_page = int(re.search('page=([0-9]+)', last_page).group(1))
            for j in range(2, last_page + 1):
                request = Request(
                    url='https://www.fxp.co.il/showthread.php?t=%s&page=%s' %
                    (i, j),
                    headers=headers)
                tree = html.fromstring(urlopen(request).read().decode('utf-8'))
                messages = tree.xpath('//div[@id=\'postlist\']//\
                    blockquote[@class=\'postcontent restore\']')
                for m in messages:
                    message_text = ' '.join(m.xpath('./text()'))
                    df = df.append({'thread': i, 'post': message_text},
                                   ignore_index=True)
        count += 1
        if verbose and (count % 100 == 0):
            print('Fetched %s threads' % (count))
            print(datetime.now().time())
        if (count % 1000 == 0):
            df.to_csv(file, encoding='utf-8')
    df.to_csv(file, encoding='utf-8')

In [ ]:
thread_content(thread_ids(5), '1to240.csv')